# Technical Assessment - Data Scientist
### Author: Nicolas Santos
### August 2022
---

### Table of Contents

- [1. Notebook Set-up](#setup)
- [2. Data Cleaning](#clean)
    - [2.1 General Data](#general)
    - [2.2 Underwriting Data](#under)
    - [2.3 Data Dictionary](#dict)
    - [2.4 Further Cleaning: Outliers](#further)
- [3. Firm Selection](#selection)
---

### 1 Notebook set-up <a class="anchor" id="setup"></a>

In [31]:
#Housekeeping 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

Formating the aesthetics of the notebook.

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%html
<style>
table {float:left}
</style>

### 2 Data Cleaning<a class="anchor" id="clean"></a>

#### 2.1 General Data<a class="anchor" id="general"></a>

On a first stage I clean the structure of the data in both workbooks. Once I have an ideal data structure to work with I can proceed to study the distributions to identify possible outliers for further cleaning.

In [21]:
#Load raw excel file
data_raw = pd.read_excel(r'C:\Users\nsant\Documents\BoE\data\Data for technical assessment.xlsx', 
                             sheet_name="Dataset 1 - General", header = 1)

data_raw.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)
data_raw.rename(columns = {'2016YE':'2016YE.0', 
                           '2017YE':'2017YE.0', 
                           '2017YE':'2017YE.0', 
                           '2018YE':'2018YE.0', 
                           '2019YE':'2019YE.0', 
                           '2020YE':'2020YE.0'}, inplace = True)
# Reshape data
data_raw = pd.wide_to_long(data_raw, stubnames = ["2016YE", "2017YE", "2018YE", "2019YE", "2020YE"], i="ID", j="Series", sep='.')

data_raw.rename(columns = {'2016YE':'v2016', 
                           '2017YE':'v2017', 
                           '2017YE':'v2017', 
                           '2018YE':'v2018', 
                           '2019YE':'v2019', 
                           '2020YE':'v2020'}, inplace = True)

#Dictionary with series name
series_dic = {0:'NWP', 1:'SCR', 2:'EoF_SCR', 3:'SCR_CR', 4:'GWP', 5:'TOT_ASSETS', 6:'TOTAL_LIAB', 7:'EXCESS'}

#Add series names to DF
data_raw = data_raw.reset_index()
for k, v in series_dic.items():
    data_raw = data_raw.replace({'Series': k}, v)

#Remove string from ID
data_raw['ID'] = data_raw['ID'].str.replace('Firm ','')

#Final reshaping
data_raw = pd.wide_to_long(data_raw, stubnames='v', i=['ID','Series'], j ='Year')
data_raw = data_raw.reset_index()
general_data = pd.pivot(data_raw, index=['ID','Year'], columns = 'Series', values='v')
general_data = general_data.reset_index()
general_data = general_data.astype({'ID':'int'})
general_data = general_data.sort_values(by=['ID', 'Year'])
display(general_data)

Series,ID,Year,EXCESS,EoF_SCR,GWP,NWP,SCR,SCR_CR,TOTAL_LIAB,TOT_ASSETS
0,1,2016,2031.035937,2230.076702,1409.484796,-13779.815629,1.085360e+03,1.979865e+00,69696.557128,71727.593066
1,1,2017,4.817920,4.817920,0.000000,0.000000,9.635840e-09,4.817920e+08,0.000000,4.817920
2,1,2018,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
3,1,2019,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
4,1,2020,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1255,325,2016,285.129083,285.129083,477.737541,240.999886,2.091815e+02,1.313433e+00,1036.142704,1321.271787
1256,325,2017,339.565646,339.565646,439.224098,252.698937,2.238009e+02,1.462014e+00,1044.558032,1384.123678
1257,325,2018,334.288033,334.288033,479.519316,332.521848,2.568627e+02,1.254034e+00,1172.729179,1507.017211
1258,325,2019,326.848756,326.848756,410.711127,294.886332,2.402959e+02,1.310660e+00,1171.754721,1498.603477


The data structure above is ideal for data wranggling. I will repeat the process for the Underwriting data before merging both datasets.

#### 2.2 Underwriting Data<a class="anchor" id="under"></a>

In [28]:
#Load raw excel file
data_raw = pd.read_excel(r'C:\Users\nsant\Documents\BoE\data\Data for technical assessment.xlsx', 
                             sheet_name="Dataset 2 - Underwriting", header = 1)

data_raw.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)
data_raw.rename(columns = {'2016YE':'2016YE.0', 
                           '2017YE':'2017YE.0', 
                           '2017YE':'2017YE.0', 
                           '2018YE':'2018YE.0', 
                           '2019YE':'2019YE.0', 
                           '2020YE':'2020YE.0'}, inplace = True)
# Reshape data
data_raw = pd.wide_to_long(data_raw, stubnames = ["2016YE", "2017YE", "2018YE", "2019YE", "2020YE"], i="ID", j="Series", sep='.')

data_raw.rename(columns = {'2016YE':'v2016', 
                           '2017YE':'v2017', 
                           '2017YE':'v2017', 
                           '2018YE':'v2018', 
                           '2019YE':'v2019', 
                           '2020YE':'v2020'}, inplace = True)

#Dictionary with series name
series_dic = {0:'GCI', 1:'GBEL', 2:'NBEL', 3:'PNCR', 4:'NER', 5:'NCR', 6:'PGCR', 7:'GER', 8:'GCR'}

#Add series names to DF
data_raw = data_raw.reset_index()
for k, v in series_dic.items():
    data_raw = data_raw.replace({'Series': k}, v)

#Remove string from ID
data_raw['ID'] = data_raw['ID'].str.replace('Firm ','')

#Final reshaping
data_raw = pd.wide_to_long(data_raw, stubnames='v', i=['ID','Series'], j ='Year')
data_raw = data_raw.reset_index()

under_data = pd.pivot(data_raw, index=['ID','Year'], columns = 'Series', values='v')
under_data = under_data.reset_index()
under_data = under_data.astype({'ID':'int'})
under_data = under_data.sort_values(by=['ID', 'Year'])
display(under_data)

Series,ID,Year,GBEL,GCI,GCR,GER,NBEL,NCR,NER,PGCR,PNCR
0,1,2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,2017,7.674468,0.046674,68.215239,56.813725,7.674468,68.215239,56.813725,11.401514,11.401514
2,1,2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,2020,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1255,325,2016,15.279072,1.198052,0.164648,0.044248,4.571224,0.000000,0.000000,0.120400,0.000000
1256,325,2017,14.482158,1.376840,0.183376,0.040154,3.792835,0.000000,0.000000,0.143222,0.000000
1257,325,2018,13.716927,3.804573,0.427933,0.037627,4.347587,0.000000,0.000000,0.390306,0.000000
1258,325,2019,13.027975,2.301670,0.274733,0.043566,3.384191,0.000000,0.000000,0.231167,0.000000


Now we have two data frames with a tidy structure that we can merge together. 

In [34]:
#Merge datasets
main_data = general_data.merge(under_data, on = ['ID', 'Year'] )
display(main_data)

Series,ID,Year,EXCESS,EoF_SCR,GWP,NWP,SCR,SCR_CR,TOTAL_LIAB,TOT_ASSETS,GBEL,GCI,GCR,GER,NBEL,NCR,NER,PGCR,PNCR
0,1,2016,2031.035937,2230.076702,1409.484796,-13779.815629,1.085360e+03,1.979865e+00,69696.557128,71727.593066,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,2017,4.817920,4.817920,0.000000,0.000000,9.635840e-09,4.817920e+08,0.000000,4.817920,7.674468,0.046674,68.215239,56.813725,7.674468,68.215239,56.813725,11.401514,11.401514
2,1,2018,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,2019,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1,2020,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1620,325,2016,285.129083,285.129083,477.737541,240.999886,2.091815e+02,1.313433e+00,1036.142704,1321.271787,15.279072,1.198052,0.164648,0.044248,4.571224,0.000000,0.000000,0.120400,0.000000
1621,325,2017,339.565646,339.565646,439.224098,252.698937,2.238009e+02,1.462014e+00,1044.558032,1384.123678,14.482158,1.376840,0.183376,0.040154,3.792835,0.000000,0.000000,0.143222,0.000000
1622,325,2018,334.288033,334.288033,479.519316,332.521848,2.568627e+02,1.254034e+00,1172.729179,1507.017211,13.716927,3.804573,0.427933,0.037627,4.347587,0.000000,0.000000,0.390306,0.000000
1623,325,2019,326.848756,326.848756,410.711127,294.886332,2.402959e+02,1.310660e+00,1171.754721,1498.603477,13.027975,2.301670,0.274733,0.043566,3.384191,0.000000,0.000000,0.231167,0.000000


With so many acronyms in the variable names it is good idea to create a dictionary for reference.


#### 2.3 Data Dictionary <a class="anchor" id="dict"></a>


- EXCESS: Excess of assets over liabilities
- EoF_SCR: EoF for SCR
- GWP: Gross written premium	
- NWP: Net written premium
- SCR: Solvency capital requirement
- SCR_CR: SCR coverage ratio
- TOTAL_LIAB: Total liabilities
- TOT_ASSETS: Total assets
- GBEL: Gross best estimate liability
- GCI: Gross claims incurred	
- GCR: Gross combined ratio
- GER: Gross expense ratio
- NBEL: Net best estimate liability
- NCR: Net combined ratio
- NER: Net expense ratio
- PGCR: Pure gross claims ratio
- PNCR: Pure net claims ratio

#### 2.4 Further Cleaning: Outliers <a class="anchor" id="further"></a>

In [35]:
#Descriptive statistics
pd.set_option('display.float_format', lambda x:'%.2f'%x)
ds = main_data.describe().round(2).T
display(ds)
pd.reset_option('display.float_format')

,count,mean,std,min,25%,50%,75%,max
Series,,,,,,,,
ID,1625.00,163.00,93.85,1.00,82.00,163.00,244.00,325.00
Year,1625.00,2018.00,1.41,2016.00,2017.00,2018.00,2019.00,2020.00
EXCESS,1625.00,524.53,2020.14,-110.79,5.26,34.13,181.99,24786.14
EoF_SCR,1625.00,526.67,2072.96,-110.79,5.22,33.29,182.02,28772.49
GWP,1625.00,920.32,3863.43,-95.42,0.00,13.71,229.07,48117.99
NWP,1625.00,744.39,3473.41,-13779.82,0.00,6.70,124.46,48117.99
SCR,1625.00,338.49,1396.45,0.00,1.44,14.18,102.75,19600.26
SCR_CR,1625.00,1191339.34,29260879.89,-1.97,1.24,1.66,2.75,963584000.00
TOTAL_LIAB,1625.00,6762.98,29092.31,-1521.33,1.41,55.81,746.50,331981.94


There seem to be several noticeable outliers in the variables of our data set. I will get rid of outliers using the Z-scores of the observations across all variables. I an observation has an outlier in at least one column it will be removed.

In [37]:
# Removing data based on Z-scores
data_clean = main_data[(np.abs(stats.zscore(main_data)) < 3).all(axis=1)]

# Re-calculating descriptive statistics
pd.set_option('display.float_format', lambda x:'%.2f'%x)
ds = data_clean.describe().round(2).T
display(ds)
pd.reset_option('display.float_format')

,count,mean,std,min,25%,50%,75%,max
Series,,,,,,,,
ID,1521.00,165.28,92.30,1.00,86.00,166.00,245.00,325.00
Year,1521.00,2018.00,1.42,2016.00,2017.00,2018.00,2019.00,2020.00
EXCESS,1521.00,241.68,654.92,-110.79,5.13,30.67,154.54,6429.02
EoF_SCR,1521.00,246.84,670.76,-110.79,4.99,30.22,153.18,6465.46
GWP,1521.00,384.19,1227.53,-95.42,0.00,10.74,173.94,11020.37
NWP,1521.00,303.68,1134.35,-2305.85,0.00,5.29,83.60,11020.37
SCR,1521.00,156.29,445.45,0.00,1.18,12.85,87.31,4105.90
SCR_CR,1521.00,5287.79,145800.65,-1.97,1.22,1.67,2.79,4181572.98
TOTAL_LIAB,1521.00,2517.10,9102.64,-1521.33,1.09,46.80,529.92,79870.40


After this process a 6.4% of the sample is dropped.

### 3 Firm Selection <a class="anchor" id="Selection"></a>

In [83]:
data2020=data_clean[data_clean['Year']==2020]
data2020=data2020[['ID', 'GWP', 'NWP', 'SCR_CR', 'GCI', 'NCR', 'TOT_ASSETS']]
data2020

Series,ID,GWP,NWP,SCR_CR,GCI,NCR,TOT_ASSETS
4,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2,21.718558,21.718558,3.900918,0.000000,0.000000,38.178685
14,3,96.416626,85.735830,1.298726,0.000000,0.000000,1754.979053
24,5,83.668353,81.273653,1.899498,373.786832,1.062102,235.481902
29,6,4452.508624,3041.876078,1.398558,178.323198,0.835525,64743.030244
...,...,...,...,...,...,...,...
1604,321,46.127035,24.632234,1.697523,3.465836,0.816870,6850.295493
1609,322,2179.242094,2029.697013,1.381756,200.694626,1.168521,5885.118080
1614,323,0.000000,0.000000,2.660207,0.000000,0.000000,13.195865
1619,324,147.947620,26.546638,1.707757,5.098240,0.780065,299.339725
